# Choosing ARIMA Orders

* Goals
  * Understand PDQ terms for ARIMA (slides)
  * Understand how to choose orders manually from ACF and PACF
  * Understand how to use automatic order selection techniques using the functions below
  
Before we can apply an ARIMA forecasting model, we need to review the components of one.<br>
ARIMA, or Autoregressive Independent Moving Average is actually a combination of 3 models:
* <strong>AR(p)</strong> Autoregression - a regression model that utilizes the dependent relationship between a current observation and observations over a previous period.
* <strong>I(d)</strong> Integration - uses differencing of observations (subtracting an observation from an observation at the previous time step) in order to make the time series stationary
* <strong>MA(q)</strong> Moving Average - a model that uses the dependency between an observation and a residual error from a moving average model applied to lagged observations.

<div class="alert alert-info"><h3>Related Functions:</h3>
<tt>
<strong>
<a href='https://www.alkaline-ml.com/pmdarima/user_guide.html#user-guide'>pmdarima.auto_arima</a></strong><font color=black>(y[,start_p,d,start_q, …])</font>&nbsp;&nbsp;&nbsp;Returns the optimal order for an ARIMA model<br>

<h3>Optional Function (see note below):</h3>
<strong>
<a href='https://www.statsmodels.org/stable/generated/statsmodels.tsa.stattools.arma_order_select_ic.html'>stattools.arma_order_select_ic</a></strong><font color=black>(y[, max_ar, …])</font>&nbsp;&nbsp;Returns information criteria for many ARMA models<br><strong>
<a href='https://www.statsmodels.org/stable/generated/statsmodels.tsa.x13.x13_arima_select_order.html'>x13.x13_arima_select_order</a></strong><font color=black>(endog[, …])</font>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Perform automatic seasonal ARIMA order identification using x12/x13 ARIMA</tt></div>

In [1]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


In [3]:
# Load a non-stationary dataset
df1 = pd.read_csv('../Data/airline_passengers.csv',index_col='Month',parse_dates=True)
df1.index.freq = 'MS'

# Load a stationary dataset
df2 = pd.read_csv('../Data/DailyTotalFemaleBirths.csv',index_col='Date',parse_dates=True)
df2.index.freq = 'D'

In [4]:
help(auto_arima)

Help on function auto_arima in module pmdarima.arima.auto:

auto_arima(y, X=None, start_p=2, d=None, start_q=2, max_p=5, max_d=2, max_q=5, start_P=1, D=None, start_Q=1, max_P=2, max_D=1, max_Q=2, max_order=5, m=1, seasonal=True, stationary=False, information_criterion='aic', alpha=0.05, test='kpss', seasonal_test='ocsb', stepwise=True, n_jobs=1, start_params=None, trend=None, method='lbfgs', maxiter=50, offset_test_args=None, seasonal_test_args=None, suppress_warnings=True, error_action='trace', trace=False, random=False, random_state=None, n_fits=10, return_valid_fits=False, out_of_sample_size=0, scoring='mse', scoring_args=None, with_intercept='auto', sarimax_kwargs=None, **fit_args)
    Automatically discover the optimal order for an ARIMA model.
    
    The auto-ARIMA process seeks to identify the most optimal
    parameters for an ``ARIMA`` model, settling on a single fitted ARIMA model.
    This process is based on the commonly-used R function,
    ``forecast::auto.arima`` [3].


In [10]:
'''
So there are a lot of parameters you can pass in to the auto Auto Arima function.

The most obvious ones should be what the starting point for each parameter is and what the max value
for each parameter is.

And the first thing you pass in is the entirety of the data set that we want to predict on

right now we're not actually concerned with a trained test split.
Instead, we're using AIC as our main criterion for judging what orders we should be using

later on when we actually run into ARIMA based model and we want to evaluate how well our forecasts will perform
on a test data set, that the model hasn't seen before, then we'll actually be concerned with a trained
test split.

And then if you want, you can also pass in some starting P and Q values. so we can say something like

start_p=0,essentially saying there was no AR component, which is unlikely given this data.
We're going to say start_q=0

then we get to choose the max value.we can say max_p=6, max_q=3,
 
And another thing we're going to do is I'm going to go ahead and say seasonal = false, since
we already ran some descriptive statistics and tests and we saw that this was a stationary data set.

So I can tell a story to not worry about trying to find higher, more complex models that fit any sort
of seasonality component.

And finally, what I'm also going to do is I'm going to set Trece equal to true and this is not true
by default.

Trece will basically show you the first couple of Arima models that auto Arima is trying to fit.


And what's really interesting here is you'll notice that we're going from P=0 all the way to max
of P=6 and Q =0 all the way to Q=3.

Now it's interesting if you think about this, if we have 6 possible values for P and 3 possible
values for Q, then at a minimum we should be getting 6X3 or 18 different models to search
for.


So something to note here is because Auto ARIMA is using AIC as its information criterion.
Remember, AIC begins to punish more complex models due to their potential to overfit the data.

So what's really nice about this auto arima function?
It's actually smart enough to understand that it's not going to need to check all the way to Max P =6
or even Max Q=3.

Eventually, what it's going to realize is even as it begins raising the order of P and Q higher and
higher, the AIC is essentially staying the same.
So at that point it's going to stop and not waste your time with trying to fit every single model in
this grid search.


So we're going to go ahead and say, let's call this stepwise fit.
Since essentially performing a step wise by stepping through the different combinations here.

And we specified traces equal to true, which will allow us to end up seeing the different models that
ordinary men tried out


'''

'''
But you'll notice what it's happening is it's fitting these various Arima models with various different
orders and then reporting back the AIC, the BIC, and then how long it took to fit this particular model.


And what's really interesting here is notice that a lot of these actually have extremely similar AIC
values.

And what happens is Auto ARIMA eventually figures out that it's no longer worth it to continue increasing
the orders of some of these more complex models all the way to your max_P=6 value instead,
since these are essentially staying the same from, it's no longer worthit to keep increasing AIC since that's 
something we're looking to minimize.

So what we realize is probably the most balanced model ends up being ARIM(1,1,1)
'''
stepwise_fit=auto_arima(df2["Births"],
                        start_p=0,
                        start_q=0,
                        max_p=6,
                        max_q=3,
                        seasonal=False,
                        trace=True)

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=2650.760, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=2565.234, Time=0.04 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=2463.584, Time=0.06 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=2648.768, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=2460.154, Time=0.11 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=2461.271, Time=0.14 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.27 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=2460.722, Time=0.11 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=2536.154, Time=0.07 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=2463.050, Time=0.34 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=2459.074, Time=0.04 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=2462.221, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=2563.261, Time=0.02 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=2460.367, Time=0.06 sec
 ARIMA(1,1,2)(0,0,0)[0]             : 

This shows a recommended (p,d,q) ARIMA Order of (1,1,1), with no seasonal_order component.

We can see how this was determined by looking at the stepwise results. The recommended order is the one with the lowest <a href='https://en.wikipedia.org/wiki/Akaike_information_criterion'>Akaike information criterion</a> or AIC score. Note that the recommended model may <em>not</em> be the one with the closest fit. The AIC score takes complexity into account, and tries to identify the best <em>forecasting</em> model.

In [6]:
'''
but we dont have to actually decide that from these results from auto_arima.

Instead, we just call stepwise_fit.summary() and it's going to give you the summary of the best performing model
that Auto ARIMA thinks you should use.

We think you should use an SARIMAX(1, 1, 1).
It reports back the number of observations, the AIC score for that particular model and then more information
as far as number of lags.

So we can see here, Auto Regression, lagging one component (ar.L1	), a moving average, also lagging one
component(ma.L1), essentially saying they're both order one, which we can see in SARIMAX(1, 1, 1).

So basically what we do now for the next step is actually create an ARIMA model using stat's models
of order (1,1,1) and then continue on of our train to split forecasting and so on.

But what's really nice is we no longer need to read these complex partial autocorrelation function plots
or auto correlation function plots.

'''
stepwise_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  365
Model:               SARIMAX(1, 1, 1)   Log Likelihood               -1226.537
Date:                Sun, 31 Oct 2021   AIC                           2459.074
Time:                        01:34:05   BIC                           2470.766
Sample:                             0   HQIC                          2463.721
                                - 365                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.1252      0.060      2.097      0.036       0.008       0.242
ma.L1         -0.9624      0.017    -56.429      0.000      -0.996      -0.929
sigma2        49.1512      3.250     15.122      0.000      42.781      55.522
===================================================================================
Ljung-Box (L1) (Q):                   0.04   Jarque-Bera (JB):                25.33
Prob(Q):                              0.84   Prob(JB):                         0.00
Heteroskedasticity (H):               0.96   Skew:                             0.57
Prob(H) (two-sided):                  0.81   Kurtosis:                         3.60
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [11]:
'''

So I'm going to assume that there's probably going to be both  AR and MA component to this more complicated
data set.
So we'll say starting points for you, start_p=0,start_q=0,

Then we going to specify some max value of P and Q i.e  max_p=4,max_q=4

As for keep in mind, it may not actually reaches max values if it sees the AIC starting to converge
to some point.

Now, remember, this is seasonal data, so we're going to want to specify that seasonal=True,

So I remember that I already know the thousands of passengers is seasonal and recall that we can perform
things like that Decky fooler test to see if we have stationary or non stationary data.
And then we can plot it out and we can do an ETS to see if there is actually seasonality in the decomposition.


And the other thing I want to do here is also trace=True,.
So I can see the trace results of the first couple of different Arima models that it tried.

And then finally, since I specified that it's seasonal, I should be choosing an M value.
So what is M? 
M is just the number of periods per season.
So that's the period for seasonal differences.
M refers to the number of periods in each season.
So M would it be 4 for quarterly data, 12 for monthly data or 1 for just annual.


OK, so that's the main thing to keep in mind when you're working with seasonal data sets is make sure
seasonal is true, which again is the default.

And you also want to make sure that you specify how many periods are there per season in case you want
Auto Arima to perform some seasonal differences.
'''

'''

And notice, what it's also doing here is that you have to order you have your lower case, p,d,q, you
and your upper case P,D,Q, so you have your Arima portion and then your seasonal portion.

So this is technically running a SARIMA model.
And the way we're going to call that and sets models is SRIAMAX.
'''

stepwise_fit=auto_arima(df1["Thousands of Passengers"],
                        start_p=0,
                        start_q=0,
                        max_p=4,
                        max_q=4,
                        seasonal=True,
                        trace=True,
                        m=12)

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(1,1,1)[12]             : AIC=1032.128, Time=0.10 sec
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=1031.508, Time=0.01 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=1020.393, Time=0.05 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=1021.003, Time=0.07 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=1020.393, Time=0.02 sec
 ARIMA(1,1,0)(2,1,0)[12]             : AIC=1019.239, Time=0.11 sec
 ARIMA(1,1,0)(2,1,1)[12]             : AIC=inf, Time=0.87 sec
 ARIMA(1,1,0)(1,1,1)[12]             : AIC=1020.493, Time=0.14 sec
 ARIMA(0,1,0)(2,1,0)[12]             : AIC=1032.120, Time=0.08 sec
 ARIMA(2,1,0)(2,1,0)[12]             : AIC=1021.120, Time=0.15 sec
 ARIMA(1,1,1)(2,1,0)[12]             : AIC=1021.032, Time=0.22 sec
 ARIMA(0,1,1)(2,1,0)[12]             : AIC=1019.178, Time=0.12 sec
 ARIMA(0,1,1)(1,1,0)[12]             : AIC=1020.425, Time=0.05 sec
 ARIMA(0,1,1)(2,1,1)[12]             : AIC=inf, Time=0.54 sec
 ARIMA(0,1,1)(1,1,1)[12]     

In [8]:
'''
the summary is going to turn back the best performing model, which happens to be
this one right here SARIMAX(0, 1, 1)x(2, 1, [], 12)	.


We can see the different orders here.
So it shows that order to for auto regression as well as moving average.

And we also get to see the seasonal components.
'''
stepwise_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                  144
Model:             SARIMAX(0, 1, 1)x(2, 1, [], 12)   Log Likelihood                -505.589
Date:                             Sun, 31 Oct 2021   AIC                           1019.178
Time:                                     01:34:08   BIC                           1030.679
Sample:                                          0   HQIC                          1023.851
                                             - 144                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -0.3634      0.074     -4.945      0.000      -0.508      -0.219
ar.S.L12      -0.1239      0.090     -1.372      0.170      -0.301       0.053
ar.S.L24       0.1911      0.107      1.783      0.075      -0.019       0.401
sigma2       130.4480     15.527      8.402      0.000     100.016     160.880
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):                 4.59
Prob(Q):                              0.92   Prob(JB):                         0.10
Heteroskedasticity (H):               2.70   Skew:                             0.15
Prob(H) (two-sided):                  0.00   Kurtosis:                         3.87
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

## OPTIONAL: statsmodels ARMA_Order_Select_IC
Statsmodels has a selection tool to find orders for ARMA models on stationary data.

In [12]:
from statsmodels.tsa.stattools import arma_order_select_ic

In [13]:
help(arma_order_select_ic)

Help on function arma_order_select_ic in module statsmodels.tsa.stattools:

arma_order_select_ic(y, max_ar=4, max_ma=2, ic='bic', trend='c', model_kw=None, fit_kw=None)
    Compute information criteria for many ARMA models.
    
    Parameters
    ----------
    y : array_like
        Array of time-series data.
    max_ar : int
        Maximum number of AR lags to use. Default 4.
    max_ma : int
        Maximum number of MA lags to use. Default 2.
    ic : str, list
        Information criteria to report. Either a single string or a list
        of different criteria is possible.
    trend : str
        The trend to use when fitting the ARMA models.
    model_kw : dict
        Keyword arguments to be passed to the ``ARMA`` model.
    fit_kw : dict
        Keyword arguments to be passed to ``ARMA.fit``.
    
    Returns
    -------
    Bunch
        Dict-like object with attribute access. Each ic is an attribute with a
        DataFrame for the results. The AR order used is the row ind

In [14]:
arma_order_select_ic(df2['Births'])

{'bic':              0            1            2
 0  2502.581666  2494.238827  2494.731525
 1  2490.780306  2484.505386  2486.223523
 2  2491.963234  2485.782753  2491.097218
 3  2496.498618  2491.061564  2496.961178
 4  2501.491891  2504.012588  2498.329743,
 'bic_min_order': (1, 1)}

In [15]:
arma_order_select_ic(df1['Thousands of Passengers'])

{'bic':              0            1            2
 0  1796.307207  1627.771967  1534.002384
 1  1437.088819  1421.627523  1425.899321
 2  1425.518037  1423.098290  1493.790622
 3  1425.191373  1413.093308  1405.250904
 4  1427.576572  1570.032620  1415.455975,
 'bic_min_order': (3, 2)}